In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import requests

In [6]:
# tool create
load_dotenv()
@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [7]:
print(multiply.invoke({'a':3, 'b':4}))

12


# tool binding

In [8]:
llm = ChatOpenAI()

In [10]:
llm_with_tools = llm.bind_tools([multiply])

In [11]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="Hello! I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 58, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BZxlTYXi2e172VWfCf95GPFcb3lQ2', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--01c6e0ec-12d7-4de6-8ac0-7776483b4e38-0', usage_metadata={'input_tokens': 58, 'output_tokens': 19, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
query = HumanMessage('What is the result of multiply(3, 1000)?')

In [13]:
messages = [query]

In [14]:
messages

[HumanMessage(content='What is the result of multiply(3, 1000)?', additional_kwargs={}, response_metadata={})]

In [15]:
result = llm_with_tools.invoke(messages)

In [16]:
messages.append(result)

In [17]:
messages

[HumanMessage(content='What is the result of multiply(3, 1000)?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_08l7R4PdiBJq6s19KfW7ykYh', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 67, 'total_tokens': 85, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BZxlsNHxqY1TlsoKwcwa73CpzGJlR', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--63a8e690-748f-41f6-8725-479fd84ce3cd-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_08l7R4PdiBJq6s19KfW7ykYh', 'type': 'tool_call'}], usage_metadata=

In [18]:
llm_with_tools.invoke('can you multiply 3 with 1000').tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 1000},
 'id': 'call_AvneVutEYT41SkNDhuKRKcED',
 'type': 'tool_call'}

In [19]:
tool_result = multiply.invoke(result.tool_calls[0])

In [20]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='call_08l7R4PdiBJq6s19KfW7ykYh')

In [21]:
messages.append(tool_result)

In [22]:
messages

[HumanMessage(content='What is the result of multiply(3, 1000)?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_08l7R4PdiBJq6s19KfW7ykYh', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 67, 'total_tokens': 85, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BZxlsNHxqY1TlsoKwcwa73CpzGJlR', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--63a8e690-748f-41f6-8725-479fd84ce3cd-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_08l7R4PdiBJq6s19KfW7ykYh', 'type': 'tool_call'}], usage_metadata=

In [23]:
llm_with_tools.invoke(messages).content

'The result of multiplying 3 by 1000 is 3000.'

# Tool Calling mini project

In [24]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/e32d832b98ee371fe9387242/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [25]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1747872002,
 'time_last_update_utc': 'Thu, 22 May 2025 00:00:02 +0000',
 'time_next_update_unix': 1747958402,
 'time_next_update_utc': 'Fri, 23 May 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.6383}

In [27]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.6383})

856.383

In [28]:
# tool binding
llm = ChatOpenAI()

In [29]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [30]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [31]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [32]:
ai_message = llm_with_tools.invoke(messages)

In [33]:
messages.append(ai_message)

In [34]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_vrfhgxl8itist8371wJDLMDe',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_rgotCCK5sT3AmUmBsGEnvm3Z',
  'type': 'tool_call'}]

In [35]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [36]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vrfhgxl8itist8371wJDLMDe', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_rgotCCK5sT3AmUmBsGEnvm3Z', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 123, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BZyRBfyXy2TiFYAVD3C0cBlzopsws', 'service_tier': 'defa

In [37]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is 0.01168. \n\nBased on this conversion factor, 10 INR is equivalent to approximately 0.1168 USD.'

In [38]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

C:\Users\neogi\AppData\Local\Temp\ipykernel_2324\627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [39]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
I'm here and ready to assist you with any questions you may have. How can I help you today?

> Finished chain.
